<a href="https://colab.research.google.com/github/sucrammal/vectari/blob/main/bag_of_words_POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00


In [ ]:
!pip install nltk

In [ ]:
# Imports
import pandas as pd
from openai import AzureOpenAI

# NLTK imports
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import *

# skl imports
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from google.colab import drive
drive.mount('/content/drive')

import string

STOPWORD_SET = {'t', 'xxxx', 'XXXX', 'xx/xx/xxxx', 'xx/xx', 'their', 'should', "doesn't", 'under', 'wasn', 'on', 'needn', 'hadn', 'out', 'against', 'to', 'ours', 'them', 'was', 'then', 'few', 'after', 'can', 'over', "weren't", 'during', 'a', 'an', 'until', 'has', 'no', 'hers', 'only', "you've", "aren't", "didn't", 'me', 'if', 'which', "wasn't", 'does', 'these', 'how', 'll', 'for', 'because', 'm', 'ma', 'won', "you'll", 'yourselves', 'haven', 'were', 'he', 'is', 'each', 'why', 'you', 'did', 'yourself', 'from', 'been', 'more', 's', "hasn't", 'above', 'with', 'o', 'below', 'aren', 'it', 'now', 'ourselves', 'so', 'here', 'do', 'up', "it's", 'most', 'i', 'himself', 'y', 'we', 'again', 'yours', 'both', 'further', "isn't", 'and', 'than', 'of', 'hasn', 'into', 'or', "should've", 'whom', 'this', 'are', 'weren', 'what', "wouldn't", 're', 'she', 'herself', "you'd", 'same', 'having', 'by', 'where', 'they', 'off', 'about', 'shouldn', 'my', 'shan', 'as', 'isn', "mustn't", 'am', 'own', 'wouldn', 'those', "haven't", 'while', 'his', "that'll", 'between', 'its', 'but', 'being', 'itself', 'be', "needn't", 'don', 'at', 'mightn', 'doing', "you're", "she's", 'down', 'just', 'him', 'ain', 'mustn', 'theirs', 'very', "shan't", 'will', "hadn't", 'through', 'couldn', 'such', "won't", 'who', 'doesn', 've', 'when', 'not', 'myself', 'there', 'had', 'nor', 'other', "don't", 'her', "shouldn't", 'd', 'too', 'any', 'our', 'in', 'that', 'all', 'didn', 'themselves', 'before', 'the', "couldn't", 'your', "mightn't", 'once', 'some', 'have'}


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import userdata

# Setting up API
# gpt_config = {
#     'model': 'gpt-4-vectari-1106preview',
#     'azure_endpoint': 'https://oai-vectari-eastus2.openai.azure.com/',
#     'api_key': userdata.get('openAPIKey'),
#     'api_version': '2023-12-01-preview'
# }

gpt_config = {
    'model': 'gpt-35-turbo',
    'azure_endpoint': 'https://oai-vectari-northcentral-us.openai.azure.com/',
    'api_key': userdata.get('openAPIKey3.5'),
    'api_version': '2024-02-15-preview'
}

print(gpt_config['azure_endpoint'])

client = AzureOpenAI(
  azure_endpoint = gpt_config['azure_endpoint'],
  api_key = gpt_config['api_key'],
  api_version = gpt_config['api_version']
)

https://oai-vectari-northcentral-us.openai.azure.com/


# **Data Preprocessing:**
I'll be using chatGPT to clean up transcripts, and then use some classic NLP techniques to clean the sentences by tokenizing them, getting rid of stopwords, and stem the words.

In [ ]:
# gpt prompt to clean up and summarize the transcript.

system = """You will receive a transcript of a phone call between a call center agent at a mortgage servicing company and a customer/borrower.
The transcription text is also low quality. These are calls about financial services and we have seen some text completely out of place.  For instance 'the first drug was fixed, and any drug after that was gonna be viral' should pretty clearly be
'the first rate was fixed, and any rate after that was going to be variable'.
Your job is to parse the text and using your natural language understanding and contextual awareness, please re-write the transcript and clean any grammar / things that do not make sense in financial use cases. After the response is generated, remove all full stops, commas, semicolons, colons, and quotation marks.
"""

In [ ]:
# instantiate input file
all_complaints = pd.read_csv("/content/drive/MyDrive/Internships and Work/Summer 2024/Vectari/data/money_transfer_complaints.csv")

In [ ]:
# extract complaints with transcripts
complaints_with_transcripts = all_complaints.loc[all_complaints["Consumer complaint narrative"].notna()]
complaints_with_transcripts.shape[0]

1497

In [ ]:
LLM_complaints = complaints_with_transcripts[['Issue']].copy()
LLM_complaints['Consumer complaint narrative'] = ""

**Process one input as a test trial**

In [ ]:
# test: LLM cleaning on 1 transcript
input_text = complaints_with_transcripts.iloc[0]["Consumer complaint narrative"]

messages = [
    {
        "role":"system",
        "content": system
    },
    {
        "role":"user",
        "content":f'"""{input_text}"""'
    }
]

response = client.chat.completions.create(
    model=gpt_config['model'],
    messages = messages
)

LLM_cleaned_sample = response.choices[0].message.content
print(LLM_cleaned_sample)

I received a phone call from someone claiming that I had been selected to receive a government grant. They said I could receive the grant through Western Union. All I had to do was send a deposit to hold the money, and I would get the deposit back with the grant. So, I sent the money and now I'm waiting to receive a tracking number. However, they told me to call another number for the Federal Reserve Bank. When I called them, they asked me to send more money. I refused and asked for a refund of the money I already sent. They told me they can't refund it because it has already been processed. I don't know what to do now to get my money back.


In [ ]:
# punctuation and lowercase.
for punctuation in string.punctuation:
    LLM_cleaned_sample = LLM_cleaned_sample.replace(punctuation, '')

print(LLM_cleaned_sample)

I received a phone call from someone claiming that I had been selected to receive a government grant They said I could receive the grant through Western Union All I had to do was send a deposit to hold the money and I would get the deposit back with the grant So I sent the money and now Im waiting to receive a tracking number However they told me to call another number for the Federal Reserve Bank When I called them they asked me to send more money I refused and asked for a refund of the money I already sent They told me they cant refund it because it has already been processed I dont know what to do now to get my money back


In [ ]:
# Then, tokenize sentence
tokenized_sample = word_tokenize(LLM_cleaned_sample)
print(tokenized_sample)

['I', 'received', 'a', 'phone', 'call', 'from', 'someone', 'claiming', 'that', 'I', 'had', 'been', 'selected', 'to', 'receive', 'a', 'government', 'grant', 'They', 'said', 'I', 'could', 'receive', 'the', 'grant', 'through', 'Western', 'Union', 'All', 'I', 'had', 'to', 'do', 'was', 'send', 'a', 'deposit', 'to', 'hold', 'the', 'money', 'and', 'I', 'would', 'get', 'the', 'deposit', 'back', 'with', 'the', 'grant', 'So', 'I', 'sent', 'the', 'money', 'and', 'now', 'Im', 'waiting', 'to', 'receive', 'a', 'tracking', 'number', 'However', 'they', 'told', 'me', 'to', 'call', 'another', 'number', 'for', 'the', 'Federal', 'Reserve', 'Bank', 'When', 'I', 'called', 'them', 'they', 'asked', 'me', 'to', 'send', 'more', 'money', 'I', 'refused', 'and', 'asked', 'for', 'a', 'refund', 'of', 'the', 'money', 'I', 'already', 'sent', 'They', 'told', 'me', 'they', 'cant', 'refund', 'it', 'because', 'it', 'has', 'already', 'been', 'processed', 'I', 'dont', 'know', 'what', 'to', 'do', 'now', 'to', 'get', 'my', 'm

In [ ]:
# Remove stopwords and stem words
stemmer = PorterStemmer()
wnl = WordNetLemmatizer()

removed_stopwords_sample = []
for word in tokenized_sample:
  if word not in STOPWORD_SET:
    word = word.lower()
    # Sometimes, stemming a word ending in 'e' will just get rid of the e. Here is a quick but rough fix
    wnl.lemmatize(word) if wnl.lemmatize(word).endswith('e') else stemmer.stem(word)
    removed_stopwords_sample.append(word)
print(removed_stopwords_sample)

['i', 'received', 'phone', 'call', 'someone', 'claiming', 'i', 'selected', 'receive', 'government', 'grant', 'they', 'said', 'i', 'could', 'receive', 'grant', 'western', 'union', 'all', 'i', 'send', 'deposit', 'hold', 'money', 'i', 'would', 'get', 'deposit', 'back', 'grant', 'so', 'i', 'sent', 'money', 'im', 'waiting', 'receive', 'tracking', 'number', 'however', 'told', 'call', 'another', 'number', 'federal', 'reserve', 'bank', 'when', 'i', 'called', 'asked', 'send', 'money', 'i', 'refused', 'asked', 'refund', 'money', 'i', 'already', 'sent', 'they', 'told', 'cant', 'refund', 'already', 'processed', 'i', 'dont', 'know', 'get', 'money', 'back']


In [ ]:
def LLM_cleanup(transcript):
  messages = [
    {
        "role":"system",
        "content": system
    },
    {
        "role":"user",
        "content":f'"""{transcript}"""'
    }
  ]

  response = client.chat.completions.create(
      model=gpt_config['model'],
      messages = messages
  )

  LLM_cleaned_sample = response.choices[0].message.content
  return LLM_cleaned_sample

In [ ]:
def NL_cleanup(LLM_cleaned_sample):
  # remove punctuation
  for punctuation in string.punctuation:
    LLM_cleaned_sample = LLM_cleaned_sample.replace(punctuation, '')

  # tokenize
  tokenized_sample = word_tokenize(LLM_cleaned_sample)

  # Remove stopwords and stem words
  stemmer = PorterStemmer()
  wnl = WordNetLemmatizer()

  removed_stopwords_sample = []
  for word in tokenized_sample:
    if word not in STOPWORD_SET:
      word = word.lower()
      # Sometimes, stemming a word ending in 'e' will just get rid of the e. Here is a quick but rough fix
      wnl.lemmatize(word) if wnl.lemmatize(word).endswith('e') else stemmer.stem(word)
      removed_stopwords_sample.append(word)

  cleaned_sentence = " ".join(removed_stopwords_sample)

  return cleaned_sentence


In [ ]:

# use the written functions!
cleaned_sample_1 = NL_cleanup(LLM_cleanup(complaints_with_transcripts.iloc[0]["Consumer complaint narrative"]))
cleaned_sample_2 = NL_cleanup(LLM_cleanup(complaints_with_transcripts.iloc[1]["Consumer complaint narrative"]))

In [ ]:
train_text_sample = [cleaned_sample_1, cleaned_sample_2]
train_label_sample = [complaints_with_transcripts.iloc[0]["Issue"], complaints_with_transcripts.iloc[1]["Issue"]]

In [ ]:
# Sample vectorization
vectorizer = CountVectorizer()

vectorizer = CountVectorizer()
vectorizer.fit(train_text_sample)
train_vect = vectorizer.transform(train_text_sample)
print(train_vect)

  (0, 7)	1
  (0, 8)	1
  (0, 11)	1
  (0, 17)	1
  (0, 18)	1
  (0, 21)	2
  (0, 23)	1
  (0, 30)	2
  (0, 32)	1
  (0, 41)	1
  (0, 44)	1
  (0, 55)	1
  (0, 60)	3
  (0, 75)	1
  (0, 78)	1
  (0, 85)	1
  (0, 86)	1
  (0, 87)	2
  (0, 89)	1
  (0, 92)	1
  (0, 101)	1
  (0, 106)	1
  (0, 116)	4
  (0, 121)	1
  (0, 123)	2
  :	:
  (1, 201)	1
  (1, 202)	1
  (1, 203)	1
  (1, 205)	3
  (1, 206)	3
  (1, 207)	3
  (1, 208)	2
  (1, 209)	1
  (1, 210)	1
  (1, 211)	1
  (1, 212)	1
  (1, 215)	2
  (1, 216)	6
  (1, 217)	1
  (1, 218)	1
  (1, 219)	1
  (1, 221)	1
  (1, 222)	1
  (1, 223)	13
  (1, 226)	3
  (1, 227)	1
  (1, 228)	1
  (1, 229)	2
  (1, 230)	2
  (1, 231)	1


**Now process ALL transcripts**

In [ ]:
cleaned_sentences = []
all_labels = complaints_with_transcripts['Issue']

for index in range(complaints_with_transcripts.shape[0]):
  cleaned_sentences.append(NL_cleanup(LLM_cleanup(complaints_with_transcripts.iloc[index]["Consumer complaint narrative"])))



In [ ]:
# @markdown  Following hiddencode blocks are hard-coded, unfactored trials.
# use chatGPT to clean up all transcripts.
# use LLM get summarized and cleaned up transcript.
for idx in range(complaints_with_transcripts.shape[0]):
  input_text = complaints_with_transcripts.iloc[idx]["Consumer complaint narrative"]

  messages = [
      {
          "role":"system",
          "content": system
      },
      {
          "role":"user",
          "content":f'"""{input_text}"""'
      }
  ]

  response = client.chat.completions.create(
      model=gpt_config['model'],
      messages = messages
  )

  LLM_complaints['Consumer complaint narrative'][idx] = response.choices[0].message.content

cleaned_complaints = complaints_with_transcripts[['Issue']].copy()
cleaned_complaints['Consumer complaint narrative'] = ""

# tokenize, stem, and remove stopwords from the transcripts
# cleaned_complaints = complaints_with_transcripts[["Issue", "Consumer complaint narrative"]]
stemmer = PorterStemmer()

for i in range(LLM_complaints.shape[0]):
  current_transcript = LLM_complaints.iloc[i]["Consumer complaint narrative"]
  tokenized = word_tokenize(current_transcript)
  cleaned = []
  for j in range(len(tokenized)):
    if tokenized[j] not in STOPWORD_SET:
      cleaned.append(stemmer.stem(tokenized[j]))

  cleaned_complaints["Consumer complaint narrative"][i] = cleaned



In [ ]:
# aggregate data via "Issue" type to build bag of words for each category.
issue_groups = cleaned_complaints.groupby("Issue")

# identify unique "Issue" labels
issue_counts = cleaned_complaints['Issue'].value_counts()
print("\nNumber of entries under each 'Issue' type:")
print(issue_counts)


Number of entries under each 'Issue' type:
Issue
Fraud or scam                            508
Other transaction issues                 469
Money was not available when promised    259
Other service issues                     130
Wrong amount charged or received          73
Incorrect/missing disclosures or info     58
Name: count, dtype: int64


In [ ]:
def train_model(train_sentences, train_labels):
  vectorizer = CountVectorizer()
  vectorizer.fit(train_sentences)
  train_vect = vectorizer.transform(train_sentences)
  print(train_vect)
  model = LogisticRegression()
  model.fit(train_vect, train_labels)
  return model, vectorizer


def predict(test_sentences, test_labels, vectorizer, model):
  test_sentences = [" ".join(t) for t in  test_sentences]
  test_vect = vectorizer.transform(test_sentences)
  preds = model.predict(test_vect)
  acc = accuracy_score(test_labels, preds)
  return preds, acc


# **Train test split**

In [ ]:
# vectorize cleaned transcripts
vectorizer = CountVectorizer()

In [ ]:
# train test split. X are features, y are labels
X = cleaned_complaints['Consumer complaint narrative']
y = cleaned_complaints['Issue']

train_sentences, test_sentences, train_labels, test = train_test_split(X, y ,random_state=104, test_size=0.25, shuffle=True)

**Note on transform:**  This step converts each sentence into a vector of token counts based on the vocabulary learned during the fit step. The result is a sparse matrix where each row represents a sentence and each column represents a token from the vocabulary. The values in the matrix are the counts of each token in the corresponding sentence. This will be useful when fitting the model.

In [ ]:
# Vectorize training data
train_sentences = [" ".join(t) for t in train_sentences]
train_labels = [l for l in train_labels]
vectorizer = CountVectorizer()
vectorizer.fit(train_sentences)
train_vect = vectorizer.transform(train_sentences)
print(train_vect)

  (5, 0)	4
  (5, 23)	1
  (5, 34)	3
  (5, 48)	7
  (5, 53)	1
  (5, 59)	1
  (5, 73)	1
  (5, 76)	1
  (5, 81)	2
  (5, 87)	2
  (5, 89)	1
  (5, 97)	1
  (5, 100)	1
  (5, 102)	1
  (5, 117)	1
  (5, 135)	1
  (5, 152)	1
  (5, 163)	1
  (5, 179)	1
  (5, 213)	1
  (5, 216)	1
  (5, 223)	1
  (5, 226)	6
  (5, 234)	1
  (5, 240)	1
  :	:
  (1121, 666)	3
  (1121, 673)	1
  (1121, 681)	1
  (1121, 707)	1
  (1121, 734)	1
  (1121, 741)	4
  (1121, 751)	1
  (1121, 775)	1
  (1121, 776)	1
  (1121, 785)	1
  (1121, 787)	2
  (1121, 810)	1
  (1121, 898)	1
  (1121, 905)	1
  (1121, 911)	1
  (1121, 934)	1
  (1121, 939)	1
  (1121, 946)	1
  (1121, 955)	3
  (1121, 982)	1
  (1121, 993)	1
  (1121, 999)	1
  (1121, 1029)	5
  (1121, 1041)	1
  (1121, 1049)	2


In [ ]:
# Combine into function to do Logistic regression training
def train_model(train_sentences, train_labels):
  train_sentences = [" ".join(t) for t in train_sentences]
  train_labels = [l for l in train_labels]
  vectorizer = CountVectorizer()
  vectorizer.fit(train_sentences)
  train_vect = vectorizer.transform(train_sentences)
  print(train_vect)
  model = LogisticRegression()
  model.fit(train_vect, train_labels)
  return model, vectorizer